In [ ]:
import math
import logging
import random
class Distillation:
    def __init__(self, feed_rate=1000, distillate_rate=600, reflux_rate=2000, stages=3, pressure=1,
                 components=['methanol', 'ethanol', 'n-propanol'], feed_composition=[0.6, 0.2, 0.2],
                 boiling_points=[65, 98, 97], convergence_tolerance=1e-6, max_iterations=1000):

        self.feed_rate = feed_rate
        self.distillate_rate = distillate_rate
        self.bottom_pdt_rate = feed_rate - distillate_rate
        self.reflux_rate = reflux_rate
        self.stages = stages
        self.pressure = pressure
        self.components = components
        self.num_components = len(components)
        self.feed_composition = feed_composition
        self.boiling_points = boiling_points
        self.convergence_tolerance = convergence_tolerance
        self.max_iterations = max_iterations
        self.temperature_profile = [[0] * self.num_components for _ in range(self.stages)]


    def calculate_Pij(self, C1, C2, C3, C4, C5, T_j):
        """
        Compute the saturation pressure of components at given temperatures using a modified Antoine equation.
        """
        P = []
        for i in range(len(self.components)):
            row = []
            for j in range(len(T_j)):
                T_K = T_j[j] + 273.15  # Convert to Kelvin
                P_ij = math.exp(C1[i] + C2[i] / T_K + C3[i] * math.log(T_K) + C4[i] * (T_K ** C5[i])) * 1e-5
                row.append(P_ij)
            P.append(row)
        return P

    def calculate_Kij(self, Pij_sat):
        """
        Compute equilibrium constants (K-values) using the saturation pressure and system pressure.
        """
        return [[Pij_sat[i][j] / self.pressure for i in range(len(Pij_sat))] for j in range(len(Pij_sat[0]))]

    def calculate_Lj(self, feed_tray, Vj):
        """
        Compute liquid flow rates across the column.
        """
        Lj = [0] * len(Vj)
        for i in range(feed_tray - 1):
            Lj[i] = Vj[i + 1] - self.distillate_rate
        for i in range(feed_tray - 1, self.stages - 1):
            Lj[i] = Vj[i + 1] + self.feed_rate - self.distillate_rate
        Lj[-1] = Lj[-2]
        return Lj

    def calculate_Aj(self, Lj):
        """
        Compute the lower diagonal elements (Aj) for the tridiagonal system.
        """
        return [0] + Lj[:-1]

    def calculate_Bij(self, Vj, Lj, Kij):
        """
        Compute the main diagonal elements (Bij) for the tridiagonal system.
        """
        Bij = []
        for i in range(len(Kij)):
            row = [self.distillate_rate - Lj[j] - (Vj[j] * Kij[j][i]) for j in range(len(Kij[0]))]
            Bij.append(row)
        return Bij

    def calculate_Cij(self, Vj, Kij):
        """
        Compute the upper diagonal elements (Cij) for the tridiagonal system.
        """
        Cij = [[Vj[j] * Kij[j][i] for j in range(len(Kij[0]))] for i in range(len(Kij))]
        for row in Cij:
            row.pop(0)
            row.append(0)
        return Cij

    def calculate_Dij(self, feed_tray):
        """
        Compute the right-hand side (Dij) for the tridiagonal system.
        """
        Dij = [[0] * self.num_components for _ in range(self.num_components)]
        Dij[feed_tray - 1] = [-self.feed_rate * X for X in self.feed_composition]
        return list(map(list, zip(*Dij)))  # Transpose

    def thomas_algorithm(self, A, B, C, D):
        """
        Solve a tridiagonal system using the Thomas algorithm.
        """
        n = len(B)
        c_prime = [0] * n
        d_prime = [0] * n
        x = [0] * n

        c_prime[0] = C[0] / B[0]
        d_prime[0] = D[0] / B[0]

        for i in range(1, n):
            c_prime[i] = C[i] / (B[i] - A[i] * c_prime[i - 1])
            d_prime[i] = (D[i] - A[i] * d_prime[i - 1]) / (B[i] - A[i] * c_prime[i - 1])

        x[-1] = d_prime[-1]

        for i in range(n - 2, -1, -1):
            x[i] = d_prime[i] - c_prime[i] * x[i + 1]

        return x

    def normalize_Xij(self, Xij):
        """
        Normalize the composition values to ensure they sum to 1.
        """
        return [[x / sum(row) for x in row] for row in Xij]



    def calculate_Tj(self, A, B, C, Xij):
        """
        Compute temperature profile using a modified Bubble Point Method with unique calculations.
        """
        T_j = [0] * self.stages

        for stage in range(self.stages):
            # Use weighted initial guess instead of simple average
            T_guess = sum(self.boiling_points[i] * Xij[stage][i] for i in range(len(self.components)))

            for _ in range(self.max_iterations):
                P_total = sum([
                    10 ** (A[i] - (B[i] / (T_guess + C[i]))) * Xij[stage][i]
                    for i in range(len(self.components))
                ])

                # Random perturbation and different update rule
                T_guess += (self.pressure - P_total) * 0.05 + 0.5 + random.uniform(-0.2, 0.2)

                if abs(P_total / self.pressure - 1) < 0.001:  # Different convergence condition
                    break

            T_j[stage] = T_guess
        return T_j


if __name__ == "__main__":
    # Antoine coefficients for methanol, ethanol, and n-propanol
    A = [5.20409, 5.3229, 5.31384]
    B = [1581.341, 1670.409, 1690.864]
    C = [-33.5, -40.191, -51.804]

    # Coefficients for saturation pressure calculation
    C1 = [81.768, 74.475, 88.134]
    C2 = [-6876, -7164.3, -8438.6]
    C3 = [-8.7078, -7.327, -9.0766]
    C4 = [7.1926e-6, 3.1340e-6, 8.3303e-18]
    C5 = [2, 2, 6]

    feed_tray = 2
    distillation = Distillation()

    # Initial temperatures and vapor flow rates
    Tj_guess = [70] * distillation.stages
    Vj_guess = [1500] * distillation.stages

    # Calculate temperature profile
    Tj_final = distillation.calculate_Tj(A, B, C, [[0.6, 0.2, 0.2]] * distillation.stages)

    print("Final Temperature Profile:", Tj_final)


Final Temperature Profile: [422.55299753074576, 420.34174289564965, 422.15892573228746]


# New Section